# Codebase API Workshop Demo: Notion

by Kieron Ong

### Setup

Let's start off by installing the Notion Python SDK, a tool that helps us call the API in a more simplified way:

In [1]:
pip install notion-client

  Obtaining dependency information for httpx>=0.15.0 from https://files.pythonhosted.org/packages/33/0d/d9ce469af019741c8999711d36b270ff992ceb1a0293f73f9f34fdf131e9/httpx-0.25.0-py3-none-any.whl.metadata
  Obtaining dependency information for httpcore<0.19.0,>=0.18.0 from https://files.pythonhosted.org/packages/ac/97/724afbb7925339f6214bf1fdb5714d1a462690466832bf8fb3fd497649f1/httpcore-0.18.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 8.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Next, let's create an integration on the Notion API. You can find out how to do that here:

https://developers.notion.com/docs/create-a-notion-integration

_Under 'Getting Started', make sure to follow the **Create your integration in Notion** and **Get your API secret** sections. That's all you'lll need!_

An integration in Notion is basically a setting that allows you to expose certain parts of your Notion pages to the Notion API. By default, you won't able to use the API without setting this up!

(Can you think of why this is a good idea from a security standpoint?)

***

### Using the Notion SDK

Now, let's initialize a client using the Notion SDK, which will allow us to interact with Notion! 

_Make sure you replace the Notion token below with your API Key (from the previous step)_

In [29]:
import os
import json
from notion_client import Client

# Replace "YOUR_NOTION_TOKEN" with your actual Notion secret token
notion_token = "YOUR_NOTION_TOKEN"

notion = Client(auth=notion_token)

### Making the Database

Now, we'll need an actual page to interact with. To start, making a copy of this Notion database:

https://glossy-mango-506.notion.site/6c8ca5b2fac24d93b23bdf4488783f39?v=2f24b2080ac145c295a7fa137fc0fa14&pvs=4

_You can duplicate it by clicking 'Duplicate' on the top right._

***

Now, we'll need to enable integration on your newly created page. Follow the instructions here to do it, under **Give your notion page permissions**:

https://developers.notion.com/docs/create-a-notion-integration

_Simplified instructions: Click the top left '...' button on the page, and click Add Connections in the dropdown menu and add the integration you created earlier_

***

### Querying the Database

Next, find the database_id for your page. Simply copy the link for the page, for e.g.:

https://www.notion.so/bdc73c253fda4438bc68d1fb68a3a3ef/?v=965042da51244e74ab6022a5ef24c749

Your database ID is the string of numbers and letters after "notion.so/" and before "/?v". So in this case: _bdc73c253fda4438bc68d1fb68a3a3ef_

With this, we can now query the database to get data from the table using this API endpoint:

https://developers.notion.com/reference/post-database-query

_Make sure to replace 'database_id' below with your own database ID!_

In [82]:
# Replace "YOUR_DATABASE_ID" with your actual database ID
database_id = "YOUR_DATABASE_ID"

results = notion.databases.query(
    **{
        "database_id": database_id,
        
    }
).get("results")

print(results)

APIResponseError: path failed validation: path.database_id should be a valid uuid, instead was `"YOUR_DATABASE_ID"`.

Wow! That's alot of data; let's do some parsing!

In [67]:
output = [{
    'name': r['properties']['Name']['title'][0]['text']['content'],
    'birthday': r['properties']['Birthday']['date']['start'],
    'favorite_cake': r['properties']['Favorite Cake']['rich_text'][0]['text']['content']
} for r in results]

print(output)

[{'name': 'Kieron Ong', 'birthday': '2000-07-16', 'favorite_cake': 'Chocolate'}, {'name': 'Eduardo Lopez', 'birthday': '2001-03-11', 'favorite_cake': 'Red Velvet'}, {'name': 'Michael Ng', 'birthday': '2001-10-10', 'favorite_cake': 'Mango'}, {'name': 'Nicole Martinez', 'birthday': '2004-06-01', 'favorite_cake': 'Fruit Tart'}, {'name': 'Tanya Bhakri', 'birthday': '2023-04-11', 'favorite_cake': 'Tortoise'}]


### Modifying the Database

Much better! Something here looks wrong; there's no way Tanya is only 6 months old. Let's try to fix that!

In Notion, each entry in a database is a **page**, so we can use this endpoint:

https://developers.notion.com/reference/patch-page

In [78]:
page_id = [r['id'] for r in results if r['properties']['Name']['title'][0]['text']['content'] == 'Tanya Bhakri'][0]

result = notion.pages.update(
    **{
        'page_id': page_id,
        'properties': {
            'Birthday': {
                'date': {
                    'start': '2001-03-11'
                },
            }
        }
    })

print(result)

{'object': 'page', 'id': '4de6aa0b-9032-48bc-91ed-00210c9833c6', 'created_time': '2023-10-28T19:54:00.000Z', 'last_edited_time': '2023-10-28T20:38:00.000Z', 'created_by': {'object': 'user', 'id': 'efe5f273-5780-4d97-afff-9fe771c072eb'}, 'last_edited_by': {'object': 'user', 'id': 'bf071854-4efb-4d9f-8c1f-429181002bd1'}, 'cover': None, 'icon': None, 'parent': {'type': 'database_id', 'database_id': 'bdc73c25-3fda-4438-bc68-d1fb68a3a3ef'}, 'archived': False, 'properties': {'Favorite Cake': {'id': 'Zune', 'type': 'rich_text', 'rich_text': [{'type': 'text', 'text': {'content': 'Tortoise', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'Tortoise', 'href': None}]}, 'Birthday': {'id': '%60Nwq', 'type': 'date', 'date': {'start': '2001-03-11', 'end': None, 'time_zone': None}}, 'Name': {'id': 'title', 'type': 'title', 'title': [{'type': 'text', 'text': {'content': 'Tanya Bhakri', 'link': N

Check your notion page; we should have just fixed it! 

### Challenge: Adding New Database Entries

Now, here's a challenge — how do we add a new entry to the database? Here's an endpoint that might help:

https://developers.notion.com/reference/post-page

In [ ]:
new_database_id = # YOUR CODE HERE
new_page = {
    # YOUR CODE HERE
}
notion.pages.create(parent={"database_id": new_database_id}, properties=new_page)